In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
with open('TrainData/train_data_scaled.pkl', 'rb') as f:
    train_data = pickle.load(f)

train_data.head()

,CustomerID,ReportedFraud,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,...,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,PolicyCombinedLimit,PolicySingleLimit
0,20065,N,2015-01-09,Multi-vehicle Collision,Rear Collision,Total Loss,Other,9,4,1404,...,2.399207,471784,MALE,JD,sales,video-games,-0.834616,-0.636247,-0.151765,-0.290132
1,37589,N,2015-02-20,Single Vehicle Collision,Side Collision,Minor Damage,Ambulance,4,3,1675,...,-0.602204,431937,FEMALE,High School,transport-moving,polo,1.021569,0.893521,-1.081375,-0.983964
2,24312,N,2015-01-14,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,5,2,1546,...,1.648854,479320,FEMALE,PhD,machine-op-inspct,exercise,-0.834616,0.893521,1.397586,1.444450
3,5493,Y,2015-01-07,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,7,4,1413,...,-1.727734,607763,FEMALE,College,exec-managerial,exercise,0.880455,-1.116315,-1.081375,-0.983964
4,7704,Y,2015-02-26,Multi-vehicle Collision,Side Collision,Major Damage,Other,4,6,1367,...,-1.477616,441783,MALE,Masters,sales,chess,-0.834616,0.893521,-0.151765,-0.290132


In [3]:
train_data['DateOfIncident'] = train_data['DateOfIncident'].astype('string')
train_data['DateOfIncident'] = pd.to_datetime(train_data['DateOfIncident'])


train_data['DateOfPolicyCoverage'] = train_data['DateOfPolicyCoverage'].astype('string')
train_data['DateOfPolicyCoverage'] = pd.to_datetime(train_data['DateOfPolicyCoverage'])

train_data['TimeDelta'] = train_data['DateOfIncident'] - train_data['DateOfPolicyCoverage']
train_data['TimeDelta'] = train_data['TimeDelta'].dt.days
train_data['TimeDelta'] = train_data['TimeDelta'].astype('int64')
train_data.drop(['DateOfIncident', 'DateOfPolicyCoverage'], axis=1, inplace=True)
train_data['TimeDelta'].head()

0    5900
1    5298
2    5318
3    8393
4    4904
Name: TimeDelta, dtype: int64

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_data['TimeDelta'] = scaler.fit_transform(train_data['TimeDelta'].values.reshape(-1,1))
train_data['TimeDelta'].head()

0    0.469348
1    0.218107
2    0.226454
3    1.509788
4    0.053673
Name: TimeDelta, dtype: float64

In [5]:
with open('models/timedelta_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [6]:
with open('TestData/test_data_scaled.pkl', 'rb') as f:
    test_data = pickle.load(f)

test_data.head()

,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,...,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,PolicyCombinedLimit,PolicySingleLimit
0,10008,2015-02-05,Multi-vehicle Collision,Front Collision,Minor Damage,Ambulance,5,2,1354,4,...,-1.477616,471704,FEMALE,High School,adm-clerical,base-jumping,1.206101,-1.148558,1.397586,1.444450
1,10010,2015-01-13,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,9,5,1383,16,...,0.148148,455810,FEMALE,MD,prof-specialty,golf,1.216956,-1.456661,-1.081375,-0.983964
2,10015,2015-01-05,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,5,2,2030,20,...,0.023090,461919,MALE,JD,other-service,movies,0.265345,0.893521,-0.151765,-0.290132
3,10020,2015-01-03,Vehicle Theft,Rear Collision,Trivial Damage,None,7,5,1449,10,...,-0.101969,600904,FEMALE,Masters,exec-managerial,video-games,1.643915,0.893521,1.397586,-0.290132
4,1003,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,5,2,1916,7,...,-1.227498,430632,FEMALE,PhD,sales,board-games,0.435405,0.893521,-1.081375,-0.983964


In [7]:
test_data.dtypes

CustomerID                  int64
DateOfIncident           category
TypeOfIncident           category
TypeOfCollission         category
SeverityOfIncident       category
AuthoritiesContacted     category
IncidentState            category
IncidentCity             category
IncidentAddress          category
IncidentTime                int64
NumberOfVehicles         category
PropertyDamage           category
BodilyInjuries           category
Witnesses                category
PoliceReport             category
AmountOfTotalClaim        float64
AmountOfInjuryClaim       float64
AmountOfPropertyClaim     float64
AmountOfVehicleDamage     float64
InsurancePolicyNumber       int64
CustomerLoyaltyPeriod     float64
DateOfPolicyCoverage     category
InsurancePolicyState     category
Policy_Deductible         float64
PolicyAnnualPremium       float64
UmbrellaLimit             float64
InsuredRelationship      category
VehicleMake              category
VehicleModel             category
VehicleYOM    

In [8]:
test_data['DateOfIncident'] = test_data['DateOfIncident'].astype('string')
test_data['DateOfIncident'] = pd.to_datetime(test_data['DateOfIncident'])


test_data['DateOfPolicyCoverage'] = test_data['DateOfPolicyCoverage'].astype('string')
test_data['DateOfPolicyCoverage'] = pd.to_datetime(test_data['DateOfPolicyCoverage'])

test_data['TimeDelta'] = test_data['DateOfIncident'] - test_data['DateOfPolicyCoverage']
test_data['TimeDelta'] = test_data['TimeDelta'].dt.days
test_data['TimeDelta'] = test_data['TimeDelta'].astype('int64')
test_data.drop(['DateOfIncident', 'DateOfPolicyCoverage'], axis=1, inplace=True)
test_data['TimeDelta'].head()

0    6115
1    1160
2    1633
3    5228
4    5282
Name: TimeDelta, dtype: int64

In [9]:
test_data['TimeDelta'] = scaler.fit_transform(test_data['TimeDelta'].values.reshape(-1,1))
test_data['TimeDelta'].head()

0    0.547906
1   -1.510277
2   -1.313805
3    0.179469
4    0.201899
Name: TimeDelta, dtype: float64

In [10]:
train_data.dtypes

CustomerID                  int64
ReportedFraud            category
TypeOfIncident           category
TypeOfCollission         category
SeverityOfIncident       category
AuthoritiesContacted     category
IncidentState            category
IncidentCity             category
IncidentAddress          category
IncidentTime                int64
NumberOfVehicles         category
PropertyDamage           category
BodilyInjuries           category
Witnesses                category
PoliceReport             category
AmountOfTotalClaim        float64
AmountOfInjuryClaim       float64
AmountOfPropertyClaim     float64
AmountOfVehicleDamage     float64
InsurancePolicyNumber       int64
CustomerLoyaltyPeriod     float64
InsurancePolicyState     category
Policy_Deductible         float64
PolicyAnnualPremium       float64
UmbrellaLimit             float64
InsuredRelationship      category
VehicleMake              category
VehicleModel             category
VehicleYOM               category
InsuredAge    

In [11]:
datatypes = pd.DataFrame(train_data.dtypes)

cat_cols = datatypes[datatypes[0] == 'category'].index
cat_cols = cat_cols.tolist()
cat_cols.remove('ReportedFraud')
cat_cols

['TypeOfIncident',
 'TypeOfCollission',
 'SeverityOfIncident',
 'AuthoritiesContacted',
 'IncidentState',
 'IncidentCity',
 'IncidentAddress',
 'NumberOfVehicles',
 'PropertyDamage',
 'BodilyInjuries',
 'Witnesses',
 'PoliceReport',
 'InsurancePolicyState',
 'InsuredRelationship',
 'VehicleMake',
 'VehicleModel',
 'VehicleYOM',
 'InsuredZipCode',
 'InsuredGender',
 'InsuredEducationLevel',
 'InsuredOccupation',
 'InsuredHobbies']

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

test_data_label_encoded = test_data.copy()
train_data_label_encoded = train_data.copy()
for col in cat_cols:
    train_data_label_encoded[col] = le.fit_transform(train_data_label_encoded[col])
    test_data_label_encoded[col] = le.transform(test_data_label_encoded[col])
train_data_label_encoded.head()

,CustomerID,ReportedFraud,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,...,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,PolicyCombinedLimit,PolicySingleLimit,TimeDelta
0,20065,N,0,1,2,3,6,3,360,19,...,577,1,3,11,18,-0.834616,-0.636247,-0.151765,-0.290132,0.469348
1,37589,N,2,2,1,0,1,2,604,14,...,29,0,2,13,14,1.021569,0.893521,-1.081375,-0.983964,0.218107
2,24312,N,0,2,0,0,2,1,490,21,...,692,0,6,6,8,-0.834616,0.893521,1.397586,1.444450,0.226454
3,5493,Y,0,2,0,0,4,3,369,12,...,809,0,1,3,8,0.880455,-1.116315,-1.081375,-0.983964,1.509788
4,7704,Y,0,2,0,3,1,5,325,3,...,160,1,5,11,5,-0.834616,0.893521,-0.151765,-0.290132,0.053673


In [13]:
train_data_label_encoded['ReportedFraud'] = train_data_label_encoded['ReportedFraud'].map({'Y': 1, 'N': 0})
train_data_label_encoded['ReportedFraud'].head()

0    0
1    0
2    0
3    1
4    1
Name: ReportedFraud, dtype: category
Categories (2, int64): [0, 1]

In [14]:
with open('TrainData/train.pkl', 'wb') as f:
    pickle.dump(train_data_label_encoded, f)

with open('TestData/test.pkl', 'wb') as f:
    pickle.dump(test_data_label_encoded, f)

In [14]:
from sklearn.model_selection import train_test_split

X = train_data_label_encoded.drop(['ReportedFraud', 'CustomerID', 'InsurancePolicyNumber'], axis=1)
y = train_data_label_encoded['ReportedFraud']

X_tr, X_test_dev, y_tr, y_test_dev = train_test_split(X, y, test_size=0.2)

In [15]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def model_scores(model, X_train, y_train, X_test, y_test):
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
    print('Cross Val Score: ', scores)
    print('Cross Val Mean: ', scores.mean())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Precision: ', precision_score(y_test, y_pred))
    print('Recall: ', recall_score(y_test, y_pred))
    print('F1 Score: ', f1_score(y_test, y_pred))
    print('ROC AUC Score: ', roc_auc_score(y_test, y_pred))
    print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))

In [16]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=100000)
model_scores(logreg, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.34003215 0.34485531 0.3392283  0.35559131 0.37972647]
Cross Val Mean:  0.3518867070385153
Accuracy:  0.7780859916782247
Precision:  0.6655132641291811
Recall:  0.36821952776005107
F1 Score:  0.47411668036154486
ROC AUC Score:  0.6495941723541984
Confusion Matrix: 
 [[3911  290]
 [ 990  577]]


In [17]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
model_scores(rf, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.75080386 0.74437299 0.76286174 0.78117458 0.77071601]
Cross Val Mean:  0.761985834499564
Accuracy:  0.9251040221914009
Precision:  0.9175864606328182
Recall:  0.795788130185067
F1 Score:  0.8523581681476419
ROC AUC Score:  0.8845639056066967
Confusion Matrix: 
 [[4089  112]
 [ 320 1247]]


In [18]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
model_scores(gb, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.68569132 0.6977492  0.70900322 0.70957361 0.71198713]
Cross Val Mean:  0.7028008940096695
Accuracy:  0.8647711511789181
Precision:  0.7628590514362057
Recall:  0.7287811104020421
F1 Score:  0.7454308093994777
ROC AUC Score:  0.8221387104021636
Confusion Matrix: 
 [[3846  355]
 [ 425 1142]]


In [19]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(max_iter=100000)
model_scores(sgd, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.03376206 0.45819936 0.0233119  0.65325825 0.47304907]
Cross Val Mean:  0.32831612657893855
Accuracy:  0.6813453536754508
Precision:  0.44520824909017387
Recall:  0.7026164645820038
F1 Score:  0.545049504950495
ROC AUC Score:  0.6880137785895023
Confusion Matrix: 
 [[2829 1372]
 [ 466 1101]]


In [20]:
from sklearn.svm import LinearSVC

#svc = LinearSVC(max_iter=100000, tol = 0.001)
#model_scores(svc, X_tr, y_tr, X_test_dev, y_test_dev)

In [21]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier(max_iter=100000, tol = 0.001)
model_scores(ridge, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.22508039 0.23311897 0.21463023 0.23652454 0.24778761]
Cross Val Mean:  0.23142834600450626
Accuracy:  0.7624826629680999
Precision:  0.6781193490054249
Recall:  0.2393107849393746
F1 Score:  0.35377358490566035
ROC AUC Score:  0.5984699604296968
Confusion Matrix: 
 [[4023  178]
 [1192  375]]


In [22]:
sgd_log = SGDClassifier(loss='log_loss')
model_scores(sgd_log, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.09807074 0.97347267 0.12138264 0.28399035 0.92759453]
Cross Val Mean:  0.48090218406355334
Accuracy:  0.7382108183079057
Precision:  0.6610169491525424
Recall:  0.07466496490108487
F1 Score:  0.1341743119266055
ROC AUC Score:  0.5301913255831299
Confusion Matrix: 
 [[4141   60]
 [1450  117]]


In [23]:
sgd_mod_hinge = SGDClassifier(loss='modified_huber')
model_scores(sgd_mod_hinge, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.3721865  0.50562701 0.58118971 0.31938858 0.01850362]
Cross Val Mean:  0.35937908234667193
Accuracy:  0.7286754507628294
Precision:  0.625
Recall:  0.003190810465858328
F1 Score:  0.006349206349206348
ROC AUC Score:  0.5012383473895585
Confusion Matrix: 
 [[4198    3]
 [1562    5]]


In [24]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(base_estimator=LogisticRegression(max_iter=100000), n_estimators=100)
model_scores(ada, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.23633441 0.26768489 0.24678457 0.25100563 0.30893001]
Cross Val Mean:  0.26214789962051155
Accuracy:  0.7612690707350902
Precision:  0.6461538461538462
Recall:  0.26802807913209953
F1 Score:  0.37889039242219213
ROC AUC Score:  0.606639604907635
Confusion Matrix: 
 [[3971  230]
 [1147  420]]


In [25]:
from sklearn.tree import DecisionTreeClassifier
ada_tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)
model_scores(ada_tree, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.67926045 0.67604502 0.70016077 0.6918745  0.71359614]
Cross Val Mean:  0.6921873747002507
Accuracy:  0.8325242718446602
Precision:  0.6826747720364742
Recall:  0.7166560306317805
F1 Score:  0.699252801992528
ROC AUC Score:  0.7961999505694014
Confusion Matrix: 
 [[3679  522]
 [ 444 1123]]


In [26]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=100000)
model_scores(mlp, X_tr, y_tr, X_test_dev, y_test_dev)

Cross Val Score:  [0.46221865 0.54019293 0.37379421 0.59694288 0.31777957]
Cross Val Mean:  0.4581856466954495
Accuracy:  0.8001040221914009
Precision:  0.7518248175182481
Recall:  0.39438417358008937
F1 Score:  0.517371285056509
ROC AUC Score:  0.6729121534408421
Confusion Matrix: 
 [[3997  204]
 [ 949  618]]


Models to use RandomForest, GradientBoosting, ADA. 

In [27]:
with open('TrainData/train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

with open('TestData/test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)